In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [2]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
import os
import torch
from datasets import load_dataset

from unsloth import FastLanguageModel
from peft import LoraConfig
from trl import SFTTrainer
import transformers

## Restart your session

In [2]:
dtype = None
max_seq_length=2048
# Load the base model
model,tokenizer = FastLanguageModel.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    max_seq_length = max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)


Unsloth: You passed in `mistralai/Mistral-7B-v0.1` and `load_in_4bit = True`.
We shall load `unsloth/mistral-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# SECTION 1.2
## Setting up Dataset
- The Llama-chat model expects prompts of the form:
[INST] {user_message} [/INST]

To train the model, we construct our training data like this:
[INST] {user_message} [/INST] {response}

In [5]:
from datasets import load_dataset

dataset_name = 'lighteval/legal_summarization'
dataset = load_dataset(dataset_name,'BillSum' , split="train")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

In [6]:
def formatting_prompts_func(batch):
    texts = [value.replace("<start>", "").replace("<end>", "").strip() for value in batch["article"]]
    return {"article": texts}

In [7]:
dataset = dataset.map(formatting_prompts_func,batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['article', 'summary'],
    num_rows: 18949
})

In [10]:
from IPython.display import Markdown
Markdown(dataset["article"][0])

SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES TO NONPROFIT ORGANIZATIONS. (a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision.

In [11]:
Markdown(dataset["summary"][0])


Shields a business entity from civil liability relating to any injury or death occurring at a facility of that entity in connection with a use of such facility by a nonprofit organization if: (1) the use occurs outside the scope of business of the business entity; (2) such injury or death occurs during a period that such facility is used by such organization; and (3) the business entity authorized the use of such facility by the organization. Makes this Act inapplicable to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including misconduct that: (1) constitutes a hate crime or a crime of violence or act of international terrorism for which the defendant has been convicted in any court; or (2) involves a sexual offense for which the defendant has been convicted in any court or misconduct for which the defendant has been found to have violated a Federal or State civil rights law. Preempts State laws to the extent that such laws are inconsistent with this Act, except State law that provides additional protection from liability. Specifies that this Act shall not be construed to supersede any Federal or State health or safety law. Makes this Act inapplicable to any civil action in a State court against a business entity in which all parties are citizens of the State if such State, citing this Act's authority and containing no other provision, enacts a statute declaring the State's election that this Act shall not apply to such action in the State.

In [13]:
alpaca_prompt = """Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["article"]
    responses      = examples["summary"]
    texts = []
    for instruction, input in zip(instructions, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)


dataset = dataset.map(formatting_prompts_func, remove_columns=[f for f in dataset.features if not f == 'text'],batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

In [29]:
Markdown(dataset["text"][20])

Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
SECTION 1. SHORT TITLE. This Act may be cited as the ``Protect and Preserve International Cultural Property Act''. SEC. 2. DEFINITION. In this Act: (1) Appropriate congressional committees.--The term ``appropriate congressional committees'' means the Committee on Foreign Affairs of the House of Representatives and the Committee on Foreign Relations of the Senate. (2) Cultural property.--The term ``cultural property'' has the meaning given in Article 1(a)-1(c) of the Hague Convention for the Protection of Cultural Property in the Event of Armed Conflict, concluded at The Hague on May 14, 1954 (Treaty Doc. 106-1(A)). SEC. 3. FINDINGS AND STATEMENT OF POLICY. (a) Findings.--Congress finds the following: (1) Protecting international cultural property is a vital part of United States cultural diplomacy, showing the respect of the United States for other cultures and the common heritage of humanity. (2) International cultural property has been lost, damaged, or destroyed due to political instability, armed conflict, natural disasters, and other threats. (3) In Egypt, recent political instability has led to the ransacking of its museums, resulting in the destruction of countless ancient artifacts that will forever leave gaps in humanity's record of the ancient Egyptian civilization. (4) In Iraq, after the fall of Saddam Hussein, thieves looted the Iraq Museum in Baghdad, resulting in the loss of approximately 15,000 items. These included ancient amulets, sculptures, ivories, and cylinder seals. Many of these items remain unrecovered. (5) In Syria, the ongoing civil war has resulted in the shelling of medieval cities, damage to five UNESCO World Heritage Sites, and the looting of museums and archaeological sites. Archaeological and historic sites and artifacts in Syria date back more than six millennia and include some of the earliest examples of writing. (6) In Iraq and Syria, the militant group ISIS/ISIL has destroyed cultural sites and artifacts, such as the Tomb of Jonah in July 2014, in an effort to eradicate ethnic and religious minorities from contested territories. Concurrently, cultural antiquities that escape demolition are looted and illicitly trafficked to help fund ISIS/ISIL's militant operations. (7) In Mali, the Al-Qaeda affiliated terrorist group Ansar Dine destroyed tombs and shrines in the ancient city of Timbuktu--a major center for trade, scholarship, and Islam in the 15th and 16th centuries--and threatened collections of ancient manuscripts. (8) In Afghanistan, the Taliban decreed that the Bamiyan Buddhas, ancient statues carved into a cliff side in central Afghanistan, were to be destroyed. In 2001 the Taliban carried out their threat and destroyed the statues, leading to worldwide condemnation. (9) In Cambodia, following the Khmer Rouge's seizure of power in 1975 the Khmer Rouge systematically destroyed many of Cambodia's Buddhist temples, desecrated statues, and destroyed Buddhist literature. The Khmer Rouge also destroyed mosques and nearly every Catholic church existing in the country. (10) In China, during the Cultural Revolution much of China's antiques were destroyed, including a large portion of old Beijing. Chinese authorities are now attempting to rebuild portions of China's lost architectural heritage. (11) In Haiti, the 2010 earthquake destroyed art, artifacts, and archives important to the people of Haiti, and partially destroyed the 17th century Haitian city of Jacmel. (12) The 2004 Indian Ocean earthquake and tsunami not only affected eleven countries, causing massive loss of life, but also damaged or destroyed libraries, archives, and UNESCO World Heritage Sites such as the Mahabalipuram in India, the Sun Temple of Koranak on the Bay of Bengal, and the Old Town of Galle and its Fortifications in Sri Lanka. (13) The destruction of these and other cultural properties represents an irreparable loss of humanity's common cultural heritage and is therefore a loss for all Americans. (14) The United States Armed Forces have played important roles in preserving and protecting cultural property. On June 23, 1943, President Franklin D. Roosevelt established the American Commission for the Protection and Salvage of Artistic and Historic Monuments in War Areas to provide expert advice to the military on the protection of cultural property. The Commission formed Monuments, Fine Arts, and Archives (MFAA) teams which became part of the Civil Affairs Division of Military Government Section of the Allied armies. The individuals serving in the MFAA were known as the ``Monuments Men'' and have been credited with securing, cataloguing, and returning hundreds of thousands works of art stolen by the Nazis during World War II. (15) The U.S. Committee of the Blue Shield was founded in 2006 to support the implementation of the 1954 Hague Convention for the Protection of Cultural Property in the Event of Armed Conflict and to coordinate with the United States military, other branches of the United States Government, and other cultural heritage nongovernmental organizations in preserving international cultural property threatened by political instability, armed conflict, natural, or other disasters. (b) Statement of Policy.--It shall be the policy of the United States to-- (1) protect and preserve international cultural property at risk of destruction due to political instability, armed conflict, or natural or other disasters; (2) protect international cultural property pursuant to its obligations under the 1954 Hague Convention for the Protection of Cultural Property in the Event of Armed Conflict and customary international law in all conflicts to which the United States is a party; (3) prevent, in accordance with existing laws, importation of cultural property pillaged, looted, or stolen during political instability, armed conflict, or natural or other disasters; and (4) ensure that existing laws and regulations, including import restrictions imposed through the Office of Foreign Asset Control (OFAC) of the Department of the Treasury, are fully implemented to prevent the trafficking in stolen or looted cultural property. SEC. 4. WHITE HOUSE COORDINATOR FOR INTERNATIONAL CULTURAL PROPERTY PROTECTION. The President shall appoint a White House Coordinator for International Cultural Property Protection. The Coordinator shall-- (1) coordinate and promote efforts to address international cultural property protection activities that involve multiple Federal agencies, including diplomatic activities, military activities, law enforcement activities, import restrictions, and the work of the Cultural Antiquities Task Force established pursuant to the Consolidated Appropriations Act, 2004 (Public Law 108-199); (2) submit to the appropriate congressional committees an annual report on interagency efforts to protect international cultural property based on information required under section 5 of this Act; (3) provide policy recommendations, if necessary; (4) resolve interagency differences in a timely, efficient, and effective manner; and (5) work and consult with domestic and international actors such as foreign governments, nongovernmental organizations, museums, educational institutions, research institutions, and the U.S. Committee of the Blue Shield on efforts to promote and protect international cultural property. SEC. 5. INFORMATION ON ACTIVITIES TO PROTECT INTERNATIONAL CULTURAL PROPERTY. (a) Information on Activities of the Department of State To Protect International Cultural Property.--The Secretary of State shall submit to the White House Coordinator for International Cultural Property Protection information on efforts of the Department of State to protect international cultural property, including-- (1) activities undertaken pursuant to the Hague Convention for the Protection of Cultural Property in the Event of Armed Conflict, including-- (A) procedures the Department has instituted to protect international cultural property at risk of destruction due to political instability, armed conflict, or natural or other disasters; and (B) actions the Department has taken to protect international cultural property in conflicts to which the United States is a party; and (2) actions the Department has taken to protect international cultural property pursuant to other cultural property protection statutes, international agreements, or policies. (b) Information on Activities of USAID To Protect International Cultural Property.--The Administrator of the United States Agency for International Development (USAID) shall submit to the White House Coordinator for International Cultural Property Protection information on efforts of USAID to protect international cultural property, including activities and coordination with other Federal agencies, international organizations, and nongovernmental organizations regarding the protection of international cultural property at risk of destruction due to political unrest, armed conflict, natural or other disasters, and USAID development programs. (c) Information on Activities of the Department of Defense To Protect International Cultural Property.--The Secretary of Defense shall submit to the White House Coordinator for International Cultural Property Protection information on efforts of the Department of Defense to protect international cultural property, including activities undertaken pursuant to the Hague Convention for the Protection of Cultural Property in the Event of Armed Conflict, other cultural protection statutes, and international agreements, including-- (1) directives, policies, and regulations the Department has instituted to protect international cultural property at risk of destruction due to political instability, armed conflict, or natural or other disasters; and (2) actions the Department has taken to protect international cultural property, including actions to avoid damage to cultural property through construction activities abroad. (d) Information on Activities of the Department of Justice To Protect International Cultural Property.--The Attorney General, in consultation with the Secretary of Homeland Security, shall submit to the White House Coordinator for International Cultural Property Protection information on efforts of the Department of Justice to protect both international cultural property and international cultural property located in, or attempted to be imported into, the United States, including activities undertaken pursuant to statutes and international agreements. Such information shall include the-- (1) statutes and regulations the Department has employed in criminal, civil, and civil forfeiture actions to prevent and interdict trafficking in stolen and smuggled cultural property, including investigations into transnational organized crime; and (2) actions the Department has taken in order to ensure the consistent and effective application of law in cases relating to both international cultural property and international cultural property located in, or attempted to be imported into, the United States. SEC. 6. AUTHORIZATION FOR FEDERAL AGENCIES TO ENGAGE IN INTERNATIONAL CULTURAL PROPERTY PROTECTION ACTIVITIES WITH THE SMITHSONIAN INSTITUTION. (a) In General.--Notwithstanding any other provision of law, the Department of State, the Department of Defense, USAID, the Department of Homeland Security, the Department of Justice, the Department of the Interior, the National Archives, the National Science Foundation, and any other agency that is involved in international cultural property protection activities are authorized to enter into agreements or memoranda of understanding with the Smithsonian Institution to temporarily engage personnel from the Smithsonian Institution for the purposes of furthering such international cultural property protection activities. (b) Salaries and Expenses.--The Federal agencies or departments specified in subsection (a) are authorized to pay the salaries and expenses of personnel from the Smithsonian Institution to assist such agencies or departments in their international cultural property protection activities, including in support of military or diplomatic missions and law enforcement efforts. SEC. 7. GRANTMAKING AUTHORIZATION FOR THE SECRETARY OF STATE FOR INTERNATIONAL CULTURAL PROPERTY PROTECTION ACTIVITIES. The Secretary of State is authorized to make grants to private individuals or organizations for the purposes of international cultural property protection activities in areas at risk of destruction due to political instability, armed conflict, or natural or other disasters. SEC. 8. EMERGENCY PROTECTION FOR SYRIAN CULTURAL PROPERTY. (a) Presidential Determination.--Notwithstanding subsection (b) of section 304 of the Convention on the Cultural Property Implementation Act (19 U.S.C. 2603) (relating to a Presidential determination that an emergency condition applies with respect to any archaeological or ethnological material of any State Party to the Convention), the President shall apply the import restrictions referred to in such section 304 with respect to any archaeological or ethnological material of Syria as if Syria were a State Party to such Convention, except that subsection (c) of such section 304 shall not apply. Such import restrictions shall take effect not later than 60 days after the date of the enactment of this Act. (b) Definitions.--In this section-- (1) the term ``archaeological or ethnological material of Syria'' means cultural property of Syria and other items of archaeological, historical, cultural, rare scientific, or religious importance unlawfully removed from Syria on or after August 18, 2011; and (2) the term ``State Party'' has the meaning given such term in section 302 of the Convention on the Cultural Property Implementation Act (19 U.S.C. 2601).

### Response:
Protect and Preserve International Cultural Property Act - Directs the President to appoint a White House Coordinator for International Cultural Property Protection who shall coordinate and promote federal agency efforts to address international cultural property protection activities. Directs the Secretary of State (Secretary in this Act), the Administrator of the United States Agency for International Development (USAID), the Attorney General (DOJ), and the Secretary of Defense (DOD) to submit to the Coordinator information on efforts to protect international cultural property. Authorizes the Department of State, DOD, USAID, the Department of Homeland Security (DHS), DOJ, the Department of the Interior, the National Archives, the National Science Foundation, and any other appropriate agency to enter into agreements with the Smithsonian Institution to engage temporarily Smithsonian personnel to assist in international cultural property protection activities. Authorizes the Secretary to make grants to private individuals or organizations for international cultural property protection activities in areas at risk of destruction due to political instability, armed conflict, or natural or other disasters. Directs the President to apply specified import restrictions with respect to any archaeological or ethnological material of Syria as if Syria were a State Party to the Convention on prohibiting and preventing the illicit import, export, and transfer of ownership of cultural property (adopted by the General Conference of the United Nations Educational, Scientific, and Cultural Organization).</s>

In [15]:
new_model = 'mistral-7b-medical-assistance'

# SECTION 1.3 Training with LORA

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=4):   0%|          | 0/18949 [00:00<?, ? examples/s]

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 18,949 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.982300
2,0.965800
3,0.915100
4,0.869100
5,0.935700
6,0.953700
7,0.842300
8,0.997100
9,0.934700
10,0.923100


In [23]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is a buisness entity", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 50 ) # optionally set temperature and do_sample=True
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.\n\n### Input:\nwhat is a buisness entity\n\n### Response:\nA business entity is a legal entity that is formed to conduct business. There are several types of business entities, including corporations, partnerships, and limited liability companies. Each type of business entity has its own set of rules and regulations that must be followed']

In [27]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what does the Human Rights Information Act Require ", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 45 ) # optionally set temperature and do_sample=True
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.\n\n### Input:\nwhat does the Human Rights Information Act Require \n\n### Response:\nThe Human Rights Information Act requires the Attorney General to establish a program to collect and disseminate information on human rights practices in foreign countries. The Act requires the Attorney General to submit to Congress an annual report on human rights practices']

In [30]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what does the term cultural property mean? ", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 45 ) # optionally set temperature and do_sample=True
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.\n\n### Input:\nwhat does the term cultural property mean? \n\n### Response:\nCultural property means any property, including any archaeological resource, historic property, or object of cultural patrimony, which is of importance to the prehistory, history, or culture of a State, people, or local']

In [31]:
model.save_pretrained_merged(new_model, tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.32 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:35<00:00,  1.11s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00001-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00002-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00003-of-00003.bin...
Done.
